In [1]:
import pyspark
import random
sc = pyspark.SparkContext(appName="Pi")

In [2]:
invitationsPath =  "invitations.txt"
meetingsPath =  "meetings.txt"
usersPath =  "users.txt"
invitationsRDD =  sc.textFile(invitationsPath)
meetingsRDD =  sc.textFile(meetingsPath)
usersRDD =  sc.textFile(usersPath)

output1 = "outPart1"
output2 = "outPart2"

In [3]:
# Part1
# meeting
# MID1034,Polito project kick-off,2023/02/07-20:40:00,90,User1000
# users
# User1000,Mario,Rossi,1988/06/01,Business

def meetingMap(l):
    mid = l.split(",")[0]
    duration = float(l.split(",")[3])
    userId = l.split(",")[4]
    return (userId, (mid, duration))

meetingsPair = meetingsRDD.map(meetingMap)

def usersMap(l):
    userId = l.split(",")[0]
    return (userId, None)

#  (None, (mid, duration))
userIdsBusiness = usersRDD\
            .filter(lambda l : l.split(",")[4] == "Business")\
            .map(usersMap)

In [4]:
meetingOrganisedByBusiness = userIdsBusiness\
                .join(meetingsPair)\
                .cache()                

In [5]:
meetingOrganisedByBusiness.collect()

[('UID1', (None, ('MID1', 60.0))),
 ('UID1', (None, ('MID2', 70.0))),
 ('UID1', (None, ('MID3', 70.0))),
 ('UID1', (None, ('MID7', 60.0))),
 ('UID3', (None, ('MID6', 100.0)))]

In [6]:
res1 = meetingOrganisedByBusiness\
    .mapValues(lambda i: (i[1][1], i[1][1], i[1][1], 1))\
    .reduceByKey(lambda t1, t2: (max(t1[0], t2[0]), min(t1[1], t2[1]), t1[2]+t2[2], t1[3]+t2[3]))\
    .mapValues(lambda i: (float(i[2])/float(i[3]), i[0], i[1]))

In [7]:
res1.collect()

[('UID1', (65.0, 70.0, 60.0)), ('UID3', (100.0, 100.0, 100.0))]

In [ ]:
# Part 2
# invitations 
# MID,UID,Accepted

In [13]:
# MID, UID
pairMidUid = meetingsPair.map(lambda i: (i[0], i[1][0]))

In [14]:
pairMidUid.collect()

[('UID1', 'MID1'),
 ('UID1', 'MID2'),
 ('UID1', 'MID3'),
 ('UID2', 'MID4'),
 ('UID2', 'MID5'),
 ('UID3', 'MID6'),
 ('UID1', 'MID7')]

In [15]:
userIdsBusiness.collect()

[('UID1', None), ('UID3', None), ('UID4', None)]

In [33]:
pairMidUidNumberOfPart = pairMidUid.join(userIdsBusiness).map(lambda i: (i[1][0], i[0]))

In [34]:
pairMidUidNumberOfPart.collect()

[('MID1', 'UID1'),
 ('MID2', 'UID1'),
 ('MID3', 'UID1'),
 ('MID7', 'UID1'),
 ('MID6', 'UID3')]

In [39]:
invites = invitationsRDD.map(lambda l: (l.split(",")[0],1))

In [48]:
invetesXmeeting = invites.join(pairMidUidNumberOfPart).map(lambda i: ((i[0],i[1][1]), i[1][0])).reduceByKey(lambda v1, v2: v1+ v2)

In [49]:
invetesXmeeting.collect()

[(('MID2', 'UID1'), 3), (('MID1', 'UID1'), 5), (('MID3', 'UID1'), 1)]

In [50]:
meetingWithZerosInvitation = pairMidUidNumberOfPart.map(lambda i: (i, 0)).subtractByKey(invetesXmeeting)

In [51]:
meetingWithZerosInvitation.collect()

[(('MID7', 'UID1'), 0), (('MID6', 'UID3'), 0)]

In [23]:
def mapSizeMeeting(i):
    if (i[1] > 20):
        return (i[0][0], (1, 0, 0))
    elif (i[1] < 5):
        return (i[0][0], (0, 0, 1))
    else:
        return (i[0][0], (0, 1, 0))

In [26]:
res2 = invetesXmeeting.union(meetingWithZerosInvitation)\
        .map(lambda i: ((i[1][0], i[0]), i[1][1]))\
        .reduceByKey(lambda v1, v2: v1+ v2)\
        .map(mapSizeMeeting)\
        .reduceByKey(lambda t1, t2: (t1[0]+t2[0], t1[1]+t2[1], t1[2]+t2[2]))

In [27]:
res2.collect()

[('U', (0, 0, 2))]